In [ ]:
import pandas as pd
import numpy as np
import h5py
import math
import random

from tokenizers.models import BPE
from tokenizers import ByteLevelBPETokenizer
from sklearn.model_selection import train_test_split

In [ ]:
samples = pd.read_csv('Reviews.csv')

In [ ]:
samples['x'] = samples['Review'].str.lower()
samples['x'] = samples['x'].str.replace(r'[^a-z\s\']', ' ', regex=True)
samples['x'] = samples['x'].str.replace(r'\'', '', regex=True)
samples['x'] = samples['x'].str.replace(r'\s+', ' ', regex=True)
samples['x']

In [ ]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(samples['x'], vocab_size=1022, min_frequency=256)

In [ ]:
vocabulary = tokenizer.get_vocab()
vocabulary = {i:v+2 for i,v in sorted(vocabulary.items(), key=lambda item: item[1])}
vocabulary.update({"[PAD]": 0, "[MASK]": 1})
vocabulary = [i for i,v in sorted(vocabulary.items(), key=lambda item: item[1])]
vocabulary_size = len(vocabulary)
vocabulary_size

In [ ]:
samples['x'] = samples['x'].apply(lambda x: tokenizer.encode(x).ids)
sequence_size = samples['x'].apply(len).max()
samples['x'] = samples['x'].apply(lambda x: (([0]*sequence_size)+x)[-sequence_size:])
samples['x'] = samples['x'].apply(lambda x: np.array(x))

sequence_size

In [ ]:
indices = [list(samples[samples['x'].apply(lambda x: t in x)].index) for t in range(vocabulary_size)]

In [ ]:
x = np.array(samples['x'].tolist()).astype(np.uint16)
y = np.array(samples['Sentiment'].tolist()).astype(np.uint8)
v = np.array([s.encode('utf-8') for s in vocabulary])

In [ ]:
file = h5py.File('preprocessed.h5', 'w')

file.create_dataset('x', data=x, maxshape=(None, x.shape[1]), chunks=True, compression='gzip')
file.create_dataset('y', data=y, maxshape=(None,), chunks=True, compression='gzip') 
file.create_dataset('v', data=v)

file.close()

In [ ]:
file.close()
file = h5py.File('preprocessed.h5', 'a')
sample_rate = .1
_indices = [random.sample(i,int(len(i)*sample_rate)) for i in indices]

In [ ]:
c = 0

for t in range(vocabulary_size):
    i = _indices[t]
    c += len(i) 

    if len(i)>0:
        _x = np.array([np.where(a == t, 1, a) for a in np.array(samples.loc[i]['x'])])
        _y = np.array(samples.loc[i]['Sentiment'])

        file['x'].resize(file['x'].shape[0] + _x.shape[0], axis=0)
        file['x'][-_x.shape[0]:] = _x

        file['y'].resize(file['y'].shape[0] + _y.shape[0], axis=0)
        file['y'][-_y.shape[0]:] = _y

    file.flush()

file.close()
c